In [1]:
# Import Dependencies
import os, re
from pytubefix.contrib.search import Search, Filter
from pytubefix.innertube import _default_clients
from IPython.display import IFrame, display, clear_output

In [2]:
def read_unique_items_from_file(file: str) -> list:
    if os.path.exists(file):
        with open(file, "r") as f:
            return list(set(url.strip() for url in f.readlines() if url.strip()))
    return []
def ask_user(prompt: str) -> bool:
    while True:
        answer = input(prompt).lower()
        if ("yes").startswith(answer.lower()):
            return True
        elif ("no").startswith(answer.lower()):
            return False
def display_video(url: str):
    display(IFrame(url, width=560, height=315, extras=["controls", "autoplay"]))

In [3]:
# Folder Name
output_path = "YouTube URL Collection"
# File Name
used_urls_filename = "Used URLs.txt"
unused_urls_filename = "Unused URLs.txt"
# Additional Dependency Configurations
_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]

In [ ]:
# YouTube Search Query
search_query = """
    
    ("Donald" OR "Trump" OR "Kamala" OR "Harris") AND ("Pennsylvania")
    
"""
search_query.replace('\n', '').strip()
# YouTube Filters
yt_filters = { "type": Filter.get_type("Video") }
# Existing URLs
used_yt_urls = read_unique_items_from_file(os.path.join(output_path, used_urls_filename))
unused_yt_urls = read_unique_items_from_file(os.path.join(output_path, unused_urls_filename))
failed_yt_urls = []
# Search YouTube Videos
print(f'...Searching...')
yt_list = Search(
    query=search_query,
    filters=yt_filters,
    use_oauth=authorize_yt,
    allow_oauth_cache=authorize_yt
)
last_yt_videos_len = len(yt_list.videos)
# Loop To Search Per Batch
while True:
    for yt in yt_list.videos:
        url = yt.watch_url.strip()
        # Skip Seen Video / URL
        if url in used_yt_urls or url in unused_yt_urls: continue
        # Reset Output Console
        clear_output(wait=True)
        # Display the Vid
        display_video(f'{yt.embed_url}?autoplay=1')
        print("") # Just New Line for Better Output
        print(f'Title: {yt.title}')
        print("") # Just New Line for Better Output
        print(f'URL If Video Failed To Load: {url}')
        print("") # Just New Line for Better Output
        if ask_user("Do you want to add this Video URL (y/n)?"): 
            # Create Video Directory
            os.makedirs(output_path, exist_ok=True)
            with open(os.path.join(output_path, used_urls_filename), "w") as file:
                used_yt_urls.append(url)
                file.write("\n".join(list(set(used_yt_urls))))
        else:
            # Create Video Directory
            os.makedirs(output_path, exist_ok=True)
            with open(os.path.join(output_path, unused_urls_filename), "w") as file:
                unused_yt_urls.append(url)
                file.write("\n".join(list(set(unused_yt_urls))))
    # Search Next Batch
    print(f'...Searching [{len(yt_list.videos)} Total Video]...')
    yt_list.get_next_results()
    yt_videos_len = len(yt_list.videos)
    # Stop If No More Videos Found
    if yt_videos_len > last_yt_videos_len:
        last_yt_videos_len = yt_videos_len
        continue
    break


Title: Trump works fry station at McDonald's in Pennsylvania

URL If Video Failed To Load: https://youtube.com/watch?v=TSMfBav1I78

